# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('eeg_features.npz') 

# Access the concatenated features from the .npz file
X_time = data['time_features']

# Close the loaded file
data.close()

#print the merged array
X_time

array([[[-1.40264838e-06, -1.29630615e-06,  2.03574174e-11, ...,
          2.03574174e-11,  3.68552716e-01,  1.69556851e+00],
        [-5.92285647e-07, -5.51680372e-07,  2.44995562e-11, ...,
          2.44995562e-11,  3.34763239e-01,  2.11152035e+00],
        [ 5.46753052e-07,  2.95335191e-07,  1.87713802e-11, ...,
          1.87713802e-11,  3.42057993e-01,  1.89587307e+00],
        [ 8.75018901e-07,  8.76244542e-07,  1.91872252e-11, ...,
          1.91872252e-11,  5.58327937e-01,  1.44576839e+00]],

       [[ 1.85753913e-06,  1.98629107e-06,  1.58275506e-11, ...,
          1.58275506e-11,  3.77039020e-01,  1.77545224e+00],
        [ 6.73217650e-07,  9.50952227e-07,  1.90147011e-11, ...,
          1.90147011e-11,  3.30172521e-01,  1.95077522e+00],
        [-1.68862467e-06, -2.20984089e-06,  2.91330360e-11, ...,
          2.91330360e-11,  2.13829446e-01,  2.86906433e+00],
        [-2.31735496e-07,  2.94672290e-07,  3.87523011e-11, ...,
          3.87523011e-11,  3.97926740e-01,  1.86624

In [3]:
data = np.load('combined_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('combined_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_time.shape, Y.shape, groups_array.shape)

(73519, 4, 13) (73519,) (73519,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [14]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=1))
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256)))
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [16]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam
import os
import csv

# Define lists to store evaluation metrics for each fold
metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_time, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_time[train_index], X_time[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

     # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks = [tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
259/259 [==============================] - 24s 26ms/step - loss: 0.3279 - accuracy: 0.8519 - val_loss: 0.2697 - val_accuracy: 0.8773
Epoch 2/200
259/259 [==============================] - 6s 22ms/step - loss: 0.2598 - accuracy: 0.8864 - val_loss: 0.2159 - val_accuracy: 0.9060
Epoch 3/200
259/259 [==============================] - 5s 18ms/step - loss: 0.2241 - accuracy: 0.9068 - val_loss: 0.2010 - val_accuracy: 0.9149
Epoch 4/200
259/259 [==============================] - 5s 20ms/step - loss: 0.2044 - accuracy: 0.9165 - val_loss: 0.1829 - val_accuracy: 0.9285
Epoch 5/200
259/259 [==============================] - 5s 19ms/step - loss: 0.1931 - accuracy: 0.9208 - val_loss: 0.1625 - val_accuracy: 0.9366
Epoch 6/200
259/259 [==============================] - 5s 18ms/step - loss: 0.1853 - accuracy: 0.9260 - val_loss: 0.1613 - val_accuracy: 0.9366
Epoch 7/200
259/259 [==============================] - 5s 20ms/step - loss: 0.1778 - accuracy: 0.9286 - val_loss: 0.1562 - val_a

In [17]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [18]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9567,0.9527,0.9668,0.9597,0.9560,0.9453,0.9668,0.9130,0.9131
1,0.9604,0.9565,0.9699,0.9631,0.9598,0.9496,0.9699,0.9204,0.9205
2,0.9570,0.9555,0.9643,0.9599,0.9565,0.9487,0.9643,0.9136,0.9137
3,0.9573,0.9544,0.9661,0.9602,0.9567,0.9473,0.9661,0.9141,0.9142
4,0.9585,0.9526,0.9704,0.9614,0.9577,0.9450,0.9704,0.9166,0.9167
5,0.9573,0.9555,0.9648,0.9601,0.9568,0.9487,0.9648,0.9142,0.9142
6,0.9587,0.9568,0.9661,0.9614,0.9581,0.9502,0.9661,0.9169,0.9169
7,0.9577,0.9526,0.9689,0.9606,0.9569,0.9450,0.9689,0.9149,0.9151
8,0.9557,0.9519,0.9655,0.9587,0.9550,0.9444,0.9655,0.9108,0.9109
9,0.9601,0.9544,0.9717,0.9629,0.9593,0.9470,0.9717,0.9198,0.9200


In [19]:
metrics_df.round(4).to_csv('4_CHANNELS_TIME_COMBINED_CNN_LSTM.csv', index = False)

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
result = pd.read_csv("4_CHANNELS_TIME_COMBINED_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9567,0.9527,0.9668,0.9597,0.9560,0.9453,0.9668,0.9130,0.9131
1,0.9604,0.9565,0.9699,0.9631,0.9598,0.9496,0.9699,0.9204,0.9205
2,0.9570,0.9555,0.9643,0.9599,0.9565,0.9487,0.9643,0.9136,0.9137
3,0.9573,0.9544,0.9661,0.9602,0.9567,0.9473,0.9661,0.9141,0.9142
4,0.9585,0.9526,0.9704,0.9614,0.9577,0.9450,0.9704,0.9166,0.9167
5,0.9573,0.9555,0.9648,0.9601,0.9568,0.9487,0.9648,0.9142,0.9142
6,0.9587,0.9568,0.9661,0.9614,0.9581,0.9502,0.9661,0.9169,0.9169
7,0.9577,0.9526,0.9689,0.9606,0.9569,0.9450,0.9689,0.9149,0.9151
8,0.9557,0.9519,0.9655,0.9587,0.9550,0.9444,0.9655,0.9108,0.9109
9,0.9601,0.9544,0.9717,0.9629,0.9593,0.9470,0.9717,0.9198,0.9200


In [22]:
(result.mean()*100).round(2)

Accuracy       95.79
Precision      95.43
Recall         96.74
F1             96.08
Sensitivity    95.73
Specificity    94.71
ROC_AUC        96.74
Kappa          91.54
MCC            91.55
dtype: float64

In [23]:
import csv
for i in range (1, 11):
    df = pd.read_csv(f'epoch_{i}.csv')
    [col] = df.columns
    split_data = df[col].str.split(';')
    csv_file = (f'epoch_{i}.csv')
    # Prepare CSV file and write headers
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
    for j in range (0, 200):
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])